# Agentic RAG with LlamaIndex


In this notebook we will experiment the tool calling capability of our LLM.

- Define a reader to read the `pdf` sample file [AraGPT2](./data/aragpt2.pdf) paper.
- Define a `splitter` to process the texts of the document.
- Set the LLM embedding and generation model ids.
- Create the engines from the Indexes and define a tool wrapper around them.
- Call the LLM with the defined tools and see the results.
- Ensure that the LLM pciks the right tool.


## Setups


In [1]:
from rich import print
from dotenv import load_dotenv

In [2]:
# load env variables
_ = load_dotenv()

In [3]:
# define some constants
GENERATION_MODEL_ID = "gpt-4o-mini"
EMBEDDING_MODEL_ID = "text-embedding-3-small"

## Load Documents


In [4]:
from llama_index.core import SimpleDirectoryReader

documents_reader = SimpleDirectoryReader(input_files=["./data/aragpt2.pdf"])
documents = documents_reader.load_data()

In [5]:
print(documents[0])

Document(
    id_='a95827cd-165c-4d83-9121-26a2792a860f',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    text='Proceedings of the Sixth Arabic Natural Language Processing Workshop , pages 196–207\nKyiv, Ukraine 
(Virtual), April 19, 2021.196ARAGPT2: Pre-T rained Transformer\nfor Arabic Language Generation\nWissam Antoun and 
Fady Baly and Hazem Hajj\nAmerican University of Beirut\n{wfa07, fbg06, hh63 }@aub.edu.lb\nAbstract\nRecently, 
pre-trained transformer-based archi-\ntectures have proven to be very efﬁcient atlanguage modeling and 
understanding, giventhat they are trained on a large enough cor-pus. Applications in language generation 
for\nArabic are still lagging in comparison to other\nNLP advances primarily due to the lack ofadvanced Arabic 
language generation models.In this paper, we develop the ﬁrst advanced\nArabic language generation model, 
AraGPT2,\ntrained from scratch on a large Arabic corpusof internet text and news articles. Our largest\nmodel, 
A\nRAGPT2- MEGA , has 1.46 billion pa-\nrameters, which makes it the largest Arabiclanguage model available. 
The\nMEGA model\nwas evaluated and showed success on different\ntasks including synthetic news generation, 
andzero-shot question answering. For text gener-\nation, our best model achieves a perplexity of\n29.8 on held-out 
Wikipedia articles. A studyconducted with human evaluators showed thesigniﬁcant success of AraGPT2-mega in 
gen-erating news articles that are difﬁcult to dis-tinguish from articles written by humans. Wethus develop and 
release an automatic discrim-inator model with a 98% percent accuracy indetecting model-generated text. The 
modelsare also publicly available\n1, hoping to encour-\nage new research directions and applicationsfor Arabic 
NLP.\n1 Introduction\nFew years ago, Natural language processing\n(NLP) was revolutionized with the 
introduction\nof multi-head self-attention transformer architec-ture ( Vaswani et al. ,2017 ). The 
transformer\nachieved superior performance compared to recur-\nrent neural networks several NLP tasks 
including\nmachine translation, sentence classiﬁcation with\n1Pretrained variants of A RAGPT2 (base, 
medium,\nlarge, mega) and discriminator are publicly available 
on\ngithub.com/aub-mind/arabert/tree/master/aragpt2BERT ( Devlin et al. ,2019 ), and ELECTRA ( Clark\net al. ,2020b
), and sentence completion with GPT-\n2(Radford et al. ,2019 ), GROVER ( Zellers et al. ,\n2019 ), and CTRL ( 
Keskar et al. ,2019 ). Recent\nworks have shown that larger models pre-trained\non larger datasets can further 
improve performancei.e. RoBERTa ( Liu et al. ,2019 ), and XLM-R ( Con-\nneau et al. ,2019 ).\nOn the other hand, 
work on Arabic language\nmodeling has mostly targeted natural language\nunderstanding (NLU) by pre-training 
transformer-\nbased models using the Masked Language Model-\ning (MLM) task i.e. A RABERT ( Antoun et al. ,\n2020a 
). In contrast, Arabic text generation or\ncausal language modeling hasn’t received much\nattention. Few works such
as hULMonA ( ElJundi\net al. ,2019 ) used next word prediction as a pre-\ntraining task in for transfer learning in
Arabic text\nclassiﬁcation. ( Khooli ,2020 ) and ( Doiron ,2020 )\nleveraged the existing GPT2 English model 
andadapted it for Arabic using text from the Arabic\nWikipedia dumps, which is sub-optimal for Arabic

In [6]:
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=32)
nodes = sentence_splitter.get_nodes_from_documents(documents)

In [7]:
print(len(nodes))
print(nodes[2])

89

TextNode(
    id_='e198b637-c172-4b4c-9c79-bcb5fff93df0',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={
        <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
            node_id='a95827cd-165c-4d83-9121-26a2792a860f',
            node_type=<ObjectType.DOCUMENT: '4'>,
            metadata={
                'page_label': '1',
                'file_name': 'aragpt2.pdf',
                'file_path': 'data/aragpt2.pdf',
                'file_type': 'application/pdf',
                'file_size': 457776,
                'creation_date': '2024-11-12',
                'last_modified_date': '2024-10-22'
            },
            hash='be467e8635c423cf5e905a9469550ed6e5194fba229387b29d1ef0cf335d08de'
        ),
        <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
            node_id='9d4dab9f-2ccf-450b-8194-917eda8d6b1d',
            node_type=<ObjectType.TEXT: '1'>,
            metadata={
                'page_label': '1',
                'file_name': 'aragpt2.pdf',
                'file_path': 'data/aragpt2.pdf',
                'file_type': 'application/pdf',
                'file_size': 457776,
                'creation_date': '2024-11-12',
                'last_modified_date': '2024-10-22'
            },
            hash='d29e9f18057bb077fc974a135476bbfa465cfbf92c45fa06545fe29f0a9eb880'
        )
    },
    text='The process of pre-\ntraining A RAGPT2, a GPT -2 transformer model\nfor the Arabic language is described.
The modelcomes in 4 size variants:\nbase (135M2),medium\n(370M), large (792M) and mega (1.46B3), which\nallows the 
exploration of A RAGPT2 in multiple ap-\nplications with different data availability and com-\nputational 
constraints. The perplexity measure is\nused to automatically evaluate A RAGPT2. Fur-\nthermore, a human-based 
evaluation is provided,\nwhich highlights the ability of A RAGPT2 to de-\nceive human evaluators. Finally, an A 
RAELEC-\nTRA ( Antoun et al. ,2020b ) based detector is devel-\n2Million Parameters\n3Billion Parameters',
    mimetype='text/plain',
    start_char_idx=3290,
    end_char_idx=3930,
    text_template='{metadata_str}\n\n{content}',
    metadata_template='{key}: {value}',
    metadata_seperator='\n'
)

## Define Vector Index


In [8]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes=nodes)

## Define Vector Tool with Filters


In [9]:
from llama_index.core.vector_stores import MetadataFilters

# define a filter to get answers from specific pages
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts([{"key": "page_label", "value": "1"}])
)

In [10]:
query = "What are the AraGPT2 model sizes?"
response = vector_query_engine.query(query)

In [11]:
print(response.response)

The AraGPT2 model comes in 4 size variants: base (135M), medium (370M), large (792M), and mega (1.46B).

In [12]:
for n in response.source_nodes:
    print(n.metadata)

{
    'page_label': '1',
    'file_name': 'aragpt2.pdf',
    'file_path': 'data/aragpt2.pdf',
    'file_type': 'application/pdf',
    'file_size': 457776,
    'creation_date': '2024-11-12',
    'last_modified_date': '2024-10-22'
}

{
    'page_label': '1',
    'file_name': 'aragpt2.pdf',
    'file_path': 'data/aragpt2.pdf',
    'file_type': 'application/pdf',
    'file_size': 457776,
    'creation_date': '2024-11-12',
    'last_modified_date': '2024-10-22'
}

In [13]:
from llama_index.core.vector_stores import FilterCondition

# defining a vectro query fn that will limit the query to a certain page
# type hints and docs are a must when defining LLM tools


def query_vector_engine(query: str, page_numbers: list[str]):
    """Perform vector search over an index.
    
    inputs:
        query (str): the string query to be embedded.
        page_numbers (list[str]): filter by set of pages. leave BLANK if we want to \
perform a vector search over all pages. otherwise, filter by the set of specified pages.
    """
    metadata_dicts = [
        {"key": "page_label", "value": page_number} for page_number in page_numbers
    ]
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )
    response = query_engine.query(query)
    return response

In [14]:
from llama_index.core.tools import FunctionTool

query_vector_engine_tool = FunctionTool.from_defaults(
    fn=query_vector_engine, name="query_vector_engine_tool"
)

In [15]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model=GENERATION_MODEL_ID)

In [16]:
query = "What are the AraGPT2 model sizes described in page number 1?"
response = llm.predict_and_call(
    tools=[query_vector_engine_tool], verbose=True, user_msg=query
)

=== Calling Function ===
Calling function: query_vector_engine_tool with args: {"query": "AraGPT2 model sizes", "page_numbers": ["1"]}
=== Function Output ===
The AraGPT2 model comes in 4 size variants: base (135M), medium (370M), large (792M), and mega (1.46B).


In [17]:
print(response.response)

The AraGPT2 model comes in 4 size variants: base (135M), medium (370M), large (792M), and mega (1.46B).

In [18]:
for n in response.source_nodes:
    print(n.metadata)

{
    'page_label': '1',
    'file_name': 'aragpt2.pdf',
    'file_path': 'data/aragpt2.pdf',
    'file_type': 'application/pdf',
    'file_size': 457776,
    'creation_date': '2024-11-12',
    'last_modified_date': '2024-10-22'
}

{
    'page_label': '1',
    'file_name': 'aragpt2.pdf',
    'file_path': 'data/aragpt2.pdf',
    'file_type': 'application/pdf',
    'file_size': 457776,
    'creation_date': '2024-11-12',
    'last_modified_date': '2024-10-22'
}

## Define Summary Tool


In [19]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex(nodes=nodes)

In [20]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize")
summary_tool = QueryEngineTool(
    query_engine=summary_query_engine,
    metadata=ToolMetadata(
        name="summary_tool",
        description="Useful for summarization questions related to the aragpt2 paper."
    ),
)

In [21]:
query = "What are the AraGPT2 model sizes described in page number 1?"
response = llm.predict_and_call(
    tools=[query_vector_engine_tool, summary_tool], verbose=True, user_msg=query
)

=== Calling Function ===
Calling function: query_vector_engine_tool with args: {"query": "AraGPT2 model sizes", "page_numbers": ["1"]}
=== Function Output ===
The AraGPT2 model comes in 4 size variants: base (135M), medium (370M), large (792M), and mega (1.46B).


In [22]:
print(response.response)

The AraGPT2 model comes in 4 size variants: base (135M), medium (370M), large (792M), and mega (1.46B).

In [23]:
query = "Summarize the AraGPT2 paper for me please."
response = llm.predict_and_call(
    tools=[query_vector_engine_tool, summary_tool], verbose=True, user_msg=query
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "AraGPT2: A Pre-trained Arabic Language Model\n\nAbstract: AraGPT2 is a pre-trained language model specifically designed for the Arabic language, based on the GPT-2 architecture. This paper presents the model's architecture, training methodology, and evaluation on various Arabic NLP tasks. The authors highlight the importance of pre-trained models for low-resource languages like Arabic and demonstrate AraGPT2's effectiveness in generating coherent and contextually relevant text.\n\n1. Introduction: The introduction discusses the challenges faced in Arabic NLP, including dialectal variations and the lack of large annotated datasets. The authors emphasize the need for a robust pre-trained model to improve performance on downstream tasks.\n\n2. Model Architecture: AraGPT2 is built on the transformer architecture, similar to GPT-2, with modifications to better suit the Arabic language. The model incorporates a lar

In [24]:
print(response.response)

AraGPT2 is a pre-trained Arabic language model based on the GPT-2 architecture. It addresses challenges in Arabic 
NLP, such as dialectal variations and limited annotated data, by utilizing a transformer-based architecture 
tailored for the Arabic language with a larger vocabulary. Trained on diverse Arabic text sources using 
unsupervised learning techniques, AraGPT2 demonstrates effectiveness in tasks like text generation, sentiment 
analysis, and question answering, surpassing existing models. This model is considered a significant advancement in
Arabic NLP, offering a strong foundation for future research and applications in the field.